In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "test_JJU_YuSangMin"

In [ ]:
!pip install -qU langchain_openai

In [ ]:
email_conversation = """
From: John (John@bikecorporation.me)
To: Kim (Kim@teddyinternational.me)
Subject: “ZENESIS” bike distribution cooperation and meeting schedule proposal
Dear Mr. Kim,

I am John, Senior Executive Director at Bike Corporation. I recently learned about your new bicycle model, "ZENESIS," through your press release. Bike Corporation is a company that leads innovation and quality in the field of bicycle manufacturing and distribution, with long-time experience and expertise in this field.

We would like to request a detailed brochure for the ZENESIS model. In particular, we need information on technical specifications, battery performance, and design aspects. This information will help us further refine our proposed distribution strategy and marketing plan.

Additionally, to discuss the possibilities for collaboration in more detail, I propose a meeting next Tuesday, January 15th, at 10:00 AM. Would it be possible to meet at your office to have this discussion?

Thank you.

Best regards,
John
Senior Executive Director
Bike Corporation
"""

### Output Parser를 사용하지 않는 경우

In [ ]:
from itertools import chain
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessageChunk
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field

prompt = PromptTemplate.from_template(
    """Please extract the important partsof the following email.

    {email_conversation}"""
)

llm = ChatOpenAI(model = 'gpt-4o', temperature = 0)

chain = prompt | llm

answer = chain.stream({"email_conversation": email_conversation})

In [ ]:
def stream_response(response, return_output = False ):

  answer =     """
    Streams the response from the AI model, processing and printing each chunk.

    This function iterates over each item in the 'response' iterable. If an item is an instance of AIMessageChunk, it extracts and prints the content.
    If the item is a string, it prints the string directly.
    Optionally, the function can return the concatenated string of all response chunks.

    Args:
    - response (iterable): An iterable of response chunks, which can be AIMessageChunk objects or strings.
    - return_output (bool, optional): If True, the function returns the concatenated response string. The default is False.

    Returns:
    - str: If `return_output` is True, the concatenated response string. Otherwise, nothing is returned.
    """
  for token in response:
    if isinstance(token, AIMessageChunk):
      answer += token.content
      print(token.content, end = "", flush= True)
    elif isinstance(token, str):
      answer += token
      print(token, end = "", flush = True)
  if return_output:
    return answer


ouptput = stream_response(answer, return_output = True)

**Important Parts of the Email:**

- **Sender:** John, Senior Executive Director at Bike Corporation
- **Recipient:** Kim at Teddy International
- **Subject:** "ZENESIS" bike distribution cooperation and meeting schedule proposal
- **Key Points:**
  - John is interested in the new bicycle model "ZENESIS."
  - Request for a detailed brochure on the ZENESIS model, focusing on technical specifications, battery performance, and design aspects.
  - Proposal for a meeting to discuss collaboration possibilities.
  - Suggested meeting date and time: Next Tuesday, January 15th, at 10:00 AM.
  - Proposed meeting location: Kim's office.

In [ ]:
answer = chain.invoke({"email_conversation": email_conversation})
print(answer)

content='Here are the important parts of the email:\n\n- **Sender:** John, Senior Executive Director at Bike Corporation\n- **Recipient:** Kim at Teddy International\n- **Subject:** "ZENESIS" bike distribution cooperation and meeting schedule proposal\n- **Request:** Detailed brochure for the ZENESIS model, focusing on technical specifications, battery performance, and design aspects.\n- **Purpose:** To refine Bike Corporation\'s proposed distribution strategy and marketing plan.\n- **Meeting Proposal:** Next Tuesday, January 15th, at 10:00 AM at Kim\'s office to discuss collaboration possibilities.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 117, 'prompt_tokens': 225, 'total_tokens': 342, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fin

### Use Output Parser

In [ ]:
class EmailSummary(BaseModel):
  person: str = Field(description="The sender of the email")
  email: str = Field(description="The email adress of the sender")
  subject: str = Field(description="The subject of the email")
  summary: str = Field(description = "A summary of the email content")
  date: str = Field(
      dedcription = "The meeting date and time mentioned in the email content"
  ),
  Is_Spam: str = Field(
      description = "Is the email spamor not. 만약 스팸이면 '스팸', 아니면 'no spam'이라고 입력해."
  )



In [ ]:
Parser = PydanticOutputParser(pydantic_object = EmailSummary)

In [ ]:
Parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"person": {"description": "The sender of the email", "title": "Person", "type": "string"}, "email": {"description": "The email adress of the sender", "title": "Email", "type": "string"}, "subject": {"description": "The subject of the email", "title": "Subject", "type": "string"}, "summary": {"description": "A summary of the email content", "title": "Summary", "type": "string"}, "date": {"dedcription": "The meeting date and time mentioned in the email content", "title": "Date", "type": "string"}}, "required": ["person", 

### Peompt 제작

In [ ]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

In [ ]:
prompt = prompt.partial(format = Parser.get_format_instructions())

In [ ]:
prompt

PromptTemplate(input_variables=['email_conversation', 'question'], input_types={}, partial_variables={'format': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"person": {"description": "The sender of the email", "title": "Person", "type": "string"}, "email": {"description": "The email adress of the sender", "title": "Email", "type": "string"}, "subject": {"description": "The subject of the email", "title": "Subject", "type": "string"}, "summary": {"description": "A summary of the email content", "title": "Summary", "type": "string"}, "date": {"dedcription": "The mee

### without parser

In [ ]:
chain = prompt | llm

In [ ]:
response = chain.stream(
    {
        "email_conversation": email_conversation,
        "question": "이메일을 요약해"
    }
)

In [ ]:
output = stream_response(response, return_output = True)

```json
{
  "person": "John",
  "email": "John@bikecorporation.me",
  "subject": "“ZENESIS” bike distribution cooperation and meeting schedule proposal",
  "summary": "John, Senior Executive Director at Bike Corporation, is interested in the ZENESIS bike model and requests a detailed brochure with technical specifications, battery performance, and design aspects. He proposes a meeting to discuss potential collaboration on Tuesday, January 15th, at 10:00 AM at Kim's office.",
  "date": "Tuesday, January 15th, 10:00 AM"
}
```

### with parser


In [ ]:
chain = prompt | llm | Parser

In [ ]:
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일을 요약해",
        "format": Parser.get_format_instructions(),
    }
)

print(response)

person='John' email='John@bikecorporation.me' subject='“ZENESIS” bike distribution cooperation and meeting schedule proposal' summary="John, Senior Executive Director at Bike Corporation, is interested in the ZENESIS bike model and requests a detailed brochure with technical specifications, battery performance, and design aspects. He proposes a meeting to discuss potential collaboration on Tuesday, January 15th, at 10:00 AM at Kim's office." date='Tuesday, January 15th, 10:00 AM'


In [ ]:
response.person

'John'